In [18]:
# 20250816

In [19]:
# https://csp.gitbook.io/langchain-for-beginners/ch13-langchain-expression-language-lcel/08.-runnablewithmessagehistory

In [20]:
# 08. RunnableWithMessageHistory

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [22]:
# Load environment variables from .env file
load_dotenv()

True

In [23]:
model = ChatGroq(model="llama3-70b-8192")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant who is proficient in {ability}. Please respond in 20 characters or less.",
        ),
        # Use conversation history as a variable, history becomes the key of MessageHistory
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # Use user input as a variable
    ]
)
runnable = prompt | model  # Create a runnable object by connecting a prompt and a model

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory

In [25]:
store = {}  # A dictionary to store session records.

In [26]:
# Function to retrieve session records based on session ID
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # If the session ID is not in the store
        # Create a new ChatMessageHistory object and save it to the store.
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # Returns the session record for the given session ID.


In [27]:
with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory Object creation
        runnable,  # Runnable object to execute
        get_session_history,  # Function to retrieve session records
        input_messages_key="input",  #Specifies the key to be processed with the latest input message
        history_messages_key="history",#Specifies the key to add the previous message.  
    )
)

In [28]:
try:
    if history_factory_config:
        _config_specs = history_factory_config
    else:
        # If not provided, then we'll use the default session_id field
        _config_specs = [
            ConfigurableFieldSpec(
                id="session_id",
                annotation=str,
                name="Session ID",
                description="Unique identifier for a session.",
                default="",
                is_shared=True,
            ),
        ]
except NameError:
    # Fallback if history_factory_config isn't defined at all
    _config_specs = [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for a session.",
            default="",
            is_shared=True,
        ),
    ]


In [29]:
with_message_history.invoke(
    # Pass the math related question "What is the meaning of cosine?" as input.
    {"ability": "math", "input": "What does cosine mean?"},
    # Pass the session ID "abc123" as configuration information.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='adjacent side / hypotenuse', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 39, 'total_tokens': 47, 'completion_time': 0.027391106, 'prompt_time': 0.012258881, 'queue_time': 0.156009667, 'total_time': 0.039649987}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'stop', 'logprobs': None}, id='run--9664a7a4-560f-475c-981e-4b37c26d01a3-0', usage_metadata={'input_tokens': 39, 'output_tokens': 8, 'total_tokens': 47})

In [30]:
# Specifies the URL of the Redis server.
# REDIS_URL = "redis://localhost:6379/0"

In [31]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # Returns a RedisChatMessageHistory object based on the session ID.
    return RedisChatMessageHistory(session_id, url=os.getenv("REDIS_URL"))


with_message_history = RunnableWithMessageHistory(
    runnable,  # executable object
    get_message_history,  # Function to get message history
    input_messages_key="input",  # Key in input message
    history_messages_key="history",
)

In [32]:
with_message_history.invoke(
    # Pass the math related question "What is the meaning of cosine?" as input.
    {"ability": "math", "input": "What does cosine mean?"},
    # Set the session ID to "redis123" as a configuration option.
    config={"configurable": {"session_id": "redis123"}},
)


AIMessage(content='cos(θ) = adjacent / hypotenuse', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 61, 'total_tokens': 72, 'completion_time': 0.034996075, 'prompt_time': 0.014635405, 'queue_time': 0.143437679, 'total_time': 0.04963148}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'stop', 'logprobs': None}, id='run--4de6c31f-3c56-4b88-8bfb-2545d58b846c-0', usage_metadata={'input_tokens': 61, 'output_tokens': 11, 'total_tokens': 72})

In [33]:
with_message_history.invoke(
    # I would like to request a Korean translation of my previous answer.
    {"ability": "math", "input": "Please translate the previous answer into Korean."},
    # Set the session ID to "foobar" as the configuration value.
    config={"configurable": {"session_id": "redis123"}},
)

AIMessage(content='cos(θ) = /.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 89, 'total_tokens': 96, 'completion_time': 0.009799746, 'prompt_time': 0.016610489, 'queue_time': 0.149110316, 'total_time': 0.026410235}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'stop', 'logprobs': None}, id='run--d4ec3c47-66de-48d9-9359-30ec3463e6d0-0', usage_metadata={'input_tokens': 89, 'output_tokens': 7, 'total_tokens': 96})

In [34]:
with_message_history.invoke(
    # I would like to request a Korean translation of my previous answer.
    {"ability": "math", "input": "Please translate the previous answer into Korean."},
    # Session ID as a setting value "redis456".
    config={"configurable": {"session_id": "redis456"}},
)

AIMessage(content=', .', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 42, 'total_tokens': 45, 'completion_time': 0.016448461, 'prompt_time': 0.01231958, 'queue_time': 0.153776163, 'total_time': 0.028768041}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'stop', 'logprobs': None}, id='run--60f59d02-92e0-41bf-95e5-f750d5b454a7-0', usage_metadata={'input_tokens': 42, 'output_tokens': 3, 'total_tokens': 45})